<a href="https://colab.research.google.com/github/goldmirr/now_tensorflow_future_pytorch/blob/main/TensorFlow_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#텐서플로우 낙서장

텐서가 필요한 이유
- 행렬로 인풋/W값 저장가능
- 그리고 Node 값 계산식이 쉬워진다.

tf.matmul() 행렬 곱셈이다.
dot과 matmul 차이
dot은 두배열의 내적곱 matmul은 두 배열의 행렬곱이다.

dot은 첫번째 배열의 3번째와 2번째 배열의 뒤에서 2번째 값이 같아야 된다. ex.) A.shape(a1,a2,<font color='red'>a3</font>) B.shape(b1,<font color='red'>b2</font>,b3)
dot 결과는 (a1,a2,b1,b3)의 4차원 텐서다.

matmul의 마지막 2개의 축으로 이루어진 행렬을 나머지 축에 따라 쌓아놓은 것이라고 생각한다. 그래서 앞의 행렬의 마지막과 뒤의 행렬 마지막에서 둘째의 값이 같아야된다. 그리고 마지막 2개 앞의 박스 수가 같아야된다.
ex.) A.shape(<font color='blue'>a1</font>,a2,<font color='red'>a3</font>) B.shape(<font color='blue'>b1</font>,<font color='red'>b2</font>,b3)

maumul 결과는 A = (2,3,4)  C = (1,5,3,2,4,6) 
 np.matmul(A,C).shape = (1, 5, 3, 2, 3, 6)

tf.zeros(형태)는 0으로 가득찬 텐서를 만들어준다.

shape
복잡한 행렬 shape 생각할때 그냥 뒤에서 부터 list 읽기 ex. (3,2,4)라면 4 크기의 list가 2개있고 그런게 3개가 있다.

type 데이터타입 정수형 int 실수는 float
tf.cast()로 데이터 캐스팅 가능하다.

(고정값) tf.constant([3,4,5])는 변하지 않는 텐서  
  (변수)   tf.Variable()는 weight 설정할때 자주 쓰고 ()안에 초기값 넣어주는거 가능 
  w.assign()으로 변수값 변경가능




In [ ]:
#간단한 수학문제(키랑 신발사이즈)


import tensorflow as tf

키 =[170, 180, 175, 160]
신발 = [260, 270, 265, 255]

# 신발 = 키 * a + b   키랑 신발사이즈랑 관련이 있다고 추정

a = tf.Variable(0.1)
b = tf.Variable(0.2)

opt = tf.keras.optimizers.Adam(learning_rate=0.1)   #경사 하강법을 통해서 W를 업데이트한다. 변수 업데이트

def 손실함수():
  예측값 = 키 * a + b
  return tf.square(260 - 예측값)         # (실제값 - 예측값)^2 = 오차값^2

for i in range(300):
  opt.minimize(손실함수 , var_list=[a,b])  #  경사 하강 실행 ,   var_list=[a,b]는 업데이트할 변수들 목록을 집어넣는다. a랑 b를 이 경우에는 업데이트해줌
  print(a,b) # (a.numpy(),b.numpy())로 하면 값만 출력

텐서플로우로 딥러닝하고싶으면 코드짜는 방법은 이렇습니다

1. 모델과 레이어 대충 디자인하기
2. model.compile에 loss함수, optimizer 넣기
3. 학습용 X와 Y데이터 준비하기 
4. X와 Y데이터 model.fit에 넣기
5. 그럼 학습해주는데 끝나면 predict 해보기

를 쭉 적고 실행하면 됩니다.  
X와 Y데이터는 각각 AI에 뭘 집어넣으면 뭘 예측하게 만들건지를 집어넣으면 됩니다. 그니까 우리모델의...

*X데이터*는 [학점, 영어성적] 
*Y데이터*는 그 학생의 합격여부를 0또는 1로  

설정하면 끝입니다. 

성능 향상을 위해서는
데이터 전처리 , 모델튜닝(레이어) 

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/gpascore.csv')  #엑셀 데이터파일 열기  데이터프레임


#*****데이터 전처리하기***** 

##데이터가 비어있는 공간(평균값을 집어넣거나 행을 삭제한다.)
#print(data.isnull().sum())   # 비어있는 데이터종류와 개수를 출력해준다.
data = data.dropna() # 데이터가 NAN이거나 빈값이면 행을 제거해준다.
# data = data.fillna(100)   빈 칸을 100으로 채워준다.

## 유용한 pandas 사용법들
# print(data['gpa'].min())
# print(data['gpa'].max())
# print(data['gpa'].count())

y데이터 = data['admit'].values

x데이터 = [ ]

for i, rows in data.iterrows():  # data.iterrows()는 data라는 데이터프레임을 가로 한줄씩(행으로) 출력해준다.
  x데이터.append([rows['gre'], rows['gpa'], rows['rank']])



import tensorflow as tf
import numpy as np


model = tf.keras.models.Sequential([                    #딥러닝 모델 만들기 
  tf.keras.layers.Dense(64, activation='tanh'),         #첫번째 레이어 만들기   (노드의개수, activation='tanh' )  개수 , 활성화 함수,   
  tf.keras.layers.Dense(128, activation='tanh'),        #두번째 레이어 만들기     레이어개수 노드 개수는 마음대로 하지만 결과가 잘 나올 때까지 실험으로 파악해야된다. 보통 2의 제곱수로 많이한다.
  tf.keras.layers.Dense(1, activation='sigmoid'),       #마지막에 올 것은 예측결과(보통 확률)이다. 결과를 몇개 출력하고 싶은지에 따라 바뀜(보통 하나의 결과).
]) 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])        # optimizer은 경사법의 learnin rate(학습률)을 변화시키면서 자동으로  ,  loss는 손실함수 확률은 binary_crossentropy , metrics는 평가요소

model.fit( np.array(x데이터), np.array(y데이터),epochs=1000  ) # x데이터는 학습데이터 y데이터는 실제 데이터(정답)  epochs는 학습횟수(데이터셋 학습횟수)

# x데이터 형태 [[380, 3.21, 3],[660, 3.67, 3],[],[]]  input이고
# y데이터 형태 [0,1,1,1]                              정답이다.



#******예측******

예측값 = model.predict( [ [750, 3.70, 3], [400, 2.2, 1] ] )

print(예측값)           # [[0.6042901 ]   [0.01291543]]

